### U.S. Bureau of Labor Statistics (BLS)

This example requests data on the number of job openings and hires in the construction industry from the BLS API.

In [1]:
# Preliminaries (Python 2.7)
import requests
import json
import config # file called config.py with my API key
import pandas as pd

The BLS API version 2 (v2) is used, which requires [registration](https://data.bls.gov/registrationEngine/). I've added a header to specify that json format data should be returned. The series_dict is a python dictionary which contains the BLS series IDs of interest as keys and their description as values. The description is used to generate column names in the resultant pandas dataframe.

In [13]:
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'
key = '?registrationkey={}'.format(config.bls_key)
headers = {'Content-type': 'application/json'}  # Request json data
series_dict = {
    'JTS23000000JOL': 'Construction Job Openings (Level, Thousands)',  # Job openings (level)
    'JTS23000000HIL': 'Construction Hires (Level, Thousands)',  # Hires (level)
}

# BLS API limits requests to ten year periods, so the request is split in two:
date_list = [('2000', '2008'), ('2009', '2017')]
df = pd.DataFrame()  # blank pandas dataframe to be filled later

In [28]:
for start, end in date_list: 
    # Parameters are the two series and the start and end year
    data = json.dumps({"seriesid":series_dict.keys(), "startyear":start, "endyear":end})
    json_data = requests.post('{}{}'.format(url, key), headers=headers, data=data).json()
    d = {} # New dictionary to be filled with data
    for series in json_data['Results']['series']:
        seriesId = series['seriesID']
        d[seriesId] = pd.DataFrame(series['data'])
        if len(series['data']) > 0:  # This if/else is to allow for series of different lengths
            # Convert BLS API dates to readable format (YYYY-MM-DD)
            d[seriesId]['date'] = pd.to_datetime(d[seriesId]['period'] + ' ' + d[seriesId]['year'])
            # Keep only date and series values
            d[seriesId] = d[seriesId].sort_values(by='date')[['date', 'value']].set_index('date')
            # Rename and identify values as floating point numbers
            d[seriesId] = d[seriesId]['value'].rename(series_dict[seriesId]).astype(float)
        else:  # If blank, leave as a blank pandas series
            d[seriesId]['date'] = pd.Series()
            d[seriesId]['value'] = pd.Series()
            d[seriesId] = d[seriesId]['value'].rename(series_dict[seriesId])
    df = df.append(pd.concat([d[k] for k in series_dict.keys()], axis=1))

In [30]:
df.tail()

,"Construction Job Openings (Level, Thousands)","Construction Hires (Level, Thousands)"
date,,
2016-12-01,140.0,400.0
2017-01-01,142.0,387.0
2017-02-01,181.0,367.0
2017-03-01,161.0,372.0
2017-04-01,203.0,378.0
